In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Inisialisasi MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Path ke folder dataset utama
DATA_PATH = r"G:\aan\Skripsi\DatasetTest2"

# List label (nama folder)
labels = os.listdir(DATA_PATH)
# Pastikan urutannya benar jika diperlukan (A-Z)
labels.sort() 

print(f"Ditemukan label: {labels}")

# List untuk menampung data dan labelnya
all_landmarks = []
all_labels = []

# =======================================================
total_file_seharusnya = 0
data_gagal_deteksi = [] # Menyimpan path file yang GAGAL terdeteksi oleh MediaPipe
data_gagal_baca = []    # Menyimpan path file yang GAGAL dibaca/ditemukan (rusak/hilang)
# =======================================================

# Iterasi melalui setiap folder label (A, B, C, ...)
for label_index, label in enumerate(labels):
    folder_path = os.path.join(DATA_PATH, label)
    
    # Memastikan itu adalah direktori
    if not os.path.isdir(folder_path):
        continue

    print(f"Memproses folder: {label}")

    # Iterasi melalui setiap gambar dalam folder
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        
        # =======================================================
        #                   Hitungan total file
        # =======================================================
        total_file_seharusnya += 1
        
        try:
            # Baca gambar
            image = cv2.imread(image_path)
            
            # =======================================================
            #                   Deteksi Gagal Baca/Rusak
            # =======================================================
            if image is None:
                print(f"Gagal membaca/File rusak: {image_path}")
                data_gagal_baca.append(image_path)
                continue
            # =======================================================

            # Konversi warna gambar dari BGR ke RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Proses gambar untuk deteksi pose
            results = pose.process(image_rgb)
            
            # Jika pose terdeteksi
            if results.pose_landmarks:
                # Ekstrak landmark dan ratakan (flatten) menjadi satu baris array
                # 33 landmarks * 4 (x,y,z,vis) = 132 fitur
                landmarks = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
                
                # Tambahkan data landmark dan labelnya
                all_landmarks.append(landmarks)
                all_labels.append(label_index) # Gunakan index (0 untuk A, 1 untuk B, dst.)
            else:
                # =======================================================
                #                Deteksi Gagal Pose
                # =======================================================
                print(f"Gagal deteksi pose di: {image_path}")
                data_gagal_deteksi.append(image_path)
                # =======================================================

        except Exception as e:
            print(f"Error saat memproses gambar {image_path}: {e}")
            data_gagal_baca.append(image_path) # Anggap error lain juga kegagalan baca/proses

# Tutup objek Pose
pose.close()

# Konversi list menjadi numpy array untuk disimpan
X = np.array(all_landmarks)
y = np.array(all_labels)

# Simpan data yang sudah diekstraksi
np.save("data_landmarks.npy", X)
np.save("data_labels.npy", y)

# =======================================================
#             Cetak Laporan Akhir
# =======================================================
print("\n" + "="*50)
print("LAPORAN EKSTRAKSI DATA")
print("="*50)
print(f"Total file yang seharusnya diproses: {total_file_seharusnya}")
print(f"Total data sukses diekstraksi: {X.shape[0]}")
print(f"Total data yang HILANG/Gagal: {total_file_seharusnya - X.shape[0]}")

print("\n--- Detail Kegagalan ---")
print(f"Jumlah file GAGAL dibaca/rusak/error: {len(data_gagal_baca)}")
if data_gagal_baca:
    print("Contoh file GAGAL baca/rusak:")
    for path in data_gagal_baca:
        print(f"  - {path}")
        
print(f"\nJumlah file GAGAL deteksi oleh MediaPipe Pose: {len(data_gagal_deteksi)}")
if data_gagal_deteksi:
    print("Contoh file GAGAL deteksi:")
    for path in data_gagal_deteksi:
        print(f"  - {path}")

# Verifikasi total kegagalan
total_kegagalan_tercatat = len(data_gagal_baca) + len(data_gagal_deteksi)
print(f"\nTotal Kegagalan Tercatat: {total_kegagalan_tercatat}")
print(f"Perbedaan antara data hilang dan tercatat: { (total_file_seharusnya - X.shape[0]) - total_kegagalan_tercatat }")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# --- Muat Data Asli ---
X = np.load('data_landmarks.npy')
y = np.load('data_labels.npy')

# --- Pembagian Data (SPLIT FIRST) ---
# Memisahkan data asli menjadi Latih, Validasi, dan Uji.
# Alokasi: 60% Latih, 20% Validasi, 20% Uji.

# Pertama, pisahkan data uji (20%) dari data utama.
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# Kedua, pisahkan sisa data (80%) menjadi data latih (60%) dan validasi (20%).
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, 
    test_size=0.25, # 0.25 dari 80% adalah 20% dari total
    random_state=42, 
    stratify=y_temp
)

print(f"Ukuran Data Latih Awal: {X_train.shape[0]}")
print(f"Ukuran Data Validasi: {X_val.shape[0]}")
print(f"Ukuran Data Uji: {X_test.shape[0]}")

print("\nMenyimpan data uji ke file...")
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)
print("Data uji berhasil disimpan.")

In [ ]:
# --- Augmentasi HANYA pada Data Latih  ---
def augment_landmarks(data, noise_level=0.005):
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

X_train_augmented = []
y_train_augmented = []

# Melakukan augmentasi hanya pada X_train
for i in range(X_train.shape[0]):
    original_sample = X_train[i]
    original_label = y_train[i]
    
    # Tambahkan data asli ke list
    X_train_augmented.append(original_sample)
    y_train_augmented.append(original_label)
    
    # Membuat 10 variasi augmented untuk setiap data asli
    for _ in range(10): 
        augmented_sample = augment_landmarks(original_sample)
        X_train_augmented.append(augmented_sample)
        y_train_augmented.append(original_label)

# Konversi list augmentasi menjadi numpy array
X_train_final = np.array(X_train_augmented)
y_train_final = np.array(y_train_augmented)

print(f"Ukuran Data Latih Setelah Augmentasi: {X_train_final.shape[0]}")


# --- Pra-pemrosesan Akhir (One-Hot Encoding) ---
num_classes = len(np.unique(y))
y_train_cat = to_categorical(y_train_final, num_classes=num_classes)
y_val_cat = to_categorical(y_val, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# --- Reshape data untuk input ke CNN 1D ---
X_train_reshaped = np.expand_dims(X_train_final, axis=2)
X_val_reshaped = np.expand_dims(X_val, axis=2)
X_test_reshaped = np.expand_dims(X_test, axis=2)

In [ ]:
# --- MEMBUAT DAN MELATIH MODEL LENET ---
def create_lenet_1d(input_shape, num_classes):
    model = Sequential([
        Conv1D(filters=6, kernel_size=5, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=16, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Buat dan compile model
lenet_model = create_lenet_1d(X_train_reshaped.shape[1:], num_classes)
lenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lenet_model.summary()

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

print("\n--- Melatih Model LeNet-5 ---")
lenet_history = lenet_model.fit(
    X_train_reshaped, y_train_cat,
    epochs=100,
    batch_size=32,
    validation_data=(X_val_reshaped, y_val_cat),
    callbacks=[early_stopping]
)

# --- MENYIMPAN MODEL ---
lenet_model.save("semaphore_lenet.h5")
print("\nModel LeNet-5 berhasil disimpan.")

# --- EVALUASI AKHIR PADA DATA UJI ---
print("\n--- Evaluasi Akhir pada Data Uji (Tak Tersentuh) ---")

loss, accuracy = lenet_model.evaluate(X_test_reshaped, y_test_cat)
print(f"Akurasi pada Data Uji: {accuracy * 100:.2f}%")
print(f"Loss pada Data Uji: {loss:.4f}")

In [ ]:
# --- MEMBUAT DAN MELATIH MODEL VGG-LIKE ---
def create_vgg_like_1d(input_shape, num_classes):
    model = Sequential([
        # Block 1
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape),
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        
        # Block 2
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        BatchNormalization(),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Buat dan compile model
vgg_model = create_vgg_like_1d(X_train_reshaped.shape[1:], num_classes)
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg_model.summary()

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

print("\n--- Melatih Model VGG-like ---")
vgg_history = vgg_model.fit(
    X_train_reshaped, y_train_cat,
    epochs=100,
    batch_size=32,
    validation_data=(X_val_reshaped, y_val_cat),
    callbacks=[early_stopping]
)

# --- MENYIMPAN MODEL ---
vgg_model.save("semaphore_vgg_like.h5")
print("\nModel VGG-like berhasil disimpan.")

# --- EVALUASI AKHIR PADA DATA UJI ---
print("\n--- Evaluasi Akhir pada Data Uji (Tak Tersentuh) ---")
loss, accuracy = vgg_model.evaluate(X_test_reshaped, y_test_cat)
print(f"Akurasi pada Data Uji: {accuracy * 100:.2f}%")
print(f"Loss pada Data Uji: {loss:.4f}")

In [ ]:
from tensorflow.keras.layers import Input, Add
from tensorflow.keras.models import Model

# --- MEMBUAT DAN MELATIH MODEL RESNET-LIKE ---
def residual_block(x, filters, kernel_size=3):
    shortcut = x
    y = Conv1D(filters, kernel_size, padding='same', activation='relu')(x)
    y = BatchNormalization()(y)
    y = Conv1D(filters, kernel_size, padding='same', activation='relu')(y)
    y = BatchNormalization()(y)
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, kernel_size=1, padding='same')(shortcut)
    y = Add()([shortcut, y])
    return y

def create_resnet_like_1d(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 5, padding='same', activation='relu')(inputs)
    x = MaxPooling1D(2)(x)
    x = residual_block(x, filters=32)
    x = residual_block(x, filters=32)
    x = MaxPooling1D(2)(x)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

# Buat dan compile model
resnet_model = create_resnet_like_1d(X_train_reshaped.shape[1:], num_classes)
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.summary()

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

print("\n--- Melatih Model ResNet-like ---")
resnet_history = resnet_model.fit(
    X_train_reshaped, y_train_cat,
    epochs=100,
    batch_size=32,
    validation_data=(X_val_reshaped, y_val_cat),
    callbacks=[early_stopping]
)

# --- MENYIMPAN MODEL ---
resnet_model.save("semaphore_resnet_like.h5")
print("\nModel ResNet-like berhasil disimpan.")

# --- EVALUASI AKHIR PADA DATA UJI ---
print("\n--- Evaluasi Akhir pada Data Uji (Tak Tersentuh) ---")
loss, accuracy = resnet_model.evaluate(X_test_reshaped, y_test_cat)
print(f"Akurasi pada Data Uji: {accuracy * 100:.2f}%")
print(f"Loss pada Data Uji: {loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Evaluasi Performa pada Data Uji
print("\n\n--- Hasil Evaluasi pada Data Uji (Tak Tersentuh) ---")
loss_lenet, acc_lenet = lenet_model.evaluate(X_test_reshaped, y_test_cat, verbose=0)
loss_vgg, acc_vgg = vgg_model.evaluate(X_test_reshaped, y_test_cat, verbose=0)
loss_resnet, acc_resnet = resnet_model.evaluate(X_test_reshaped, y_test_cat, verbose=0)

print(f"LeNet-5      -> Akurasi: {acc_lenet*100:.2f}%, Loss: {loss_lenet:.4f}")
print(f"VGG-like     -> Akurasi: {acc_vgg*100:.2f}%, Loss: {loss_vgg:.4f}")
print(f"ResNet-like  -> Akurasi: {acc_resnet*100:.2f}%, Loss: {loss_resnet:.4f}")

# Perbandingan Konfigurasi Model (Jumlah Parameter)
print("\n--- Perbandingan Konfigurasi (Kompleksitas Model) ---")
print(f"LeNet-5      -> Total Parameter: {lenet_model.count_params()}")
print(f"VGG-like     -> Total Parameter: {vgg_model.count_params()}")
print(f"ResNet-like  -> Total Parameter: {resnet_model.count_params()}")

# Visualisasi Kurva Pembelajaran
print("\n--- Menampilkan Grafik Perbandingan Kurva Pembelajaran ---")
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(18, 6))

# Grafik Akurasi
plt.subplot(1, 2, 1)
plt.plot(lenet_history.history['val_accuracy'], label='LeNet Validation Accuracy', linestyle='--')
plt.plot(vgg_history.history['val_accuracy'], label='VGG Validation Accuracy', linestyle='--')
plt.plot(resnet_history.history['val_accuracy'], label='ResNet Validation Accuracy', linestyle='--')
plt.plot(lenet_history.history['accuracy'], label='LeNet Training Accuracy')
plt.plot(vgg_history.history['accuracy'], label='VGG Training Accuracy')
plt.plot(resnet_history.history['accuracy'], label='ResNet Training Accuracy')
plt.title('Perbandingan Akurasi Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()

# Grafik Loss
plt.subplot(1, 2, 2)
plt.plot(lenet_history.history['val_loss'], label='LeNet Validation Loss', linestyle='--')
plt.plot(vgg_history.history['val_loss'], label='VGG Validation Loss', linestyle='--')
plt.plot(resnet_history.history['val_loss'], label='ResNet Validation Loss', linestyle='--')
plt.plot(lenet_history.history['loss'], label='LeNet Training Loss')
plt.plot(vgg_history.history['loss'], label='VGG Training Loss')
plt.plot(resnet_history.history['loss'], label='ResNet Training Loss')
plt.title('Perbandingan Loss Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(20, 8))

# Grafik Akurasi
plt.subplot(1, 2, 1)
plt.plot(lenet_history.history['val_accuracy'], label='LeNet Validation Accuracy', linestyle='--')
plt.plot(lenet_history.history['accuracy'], label='LeNet Training Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()

# Grafik Loss
plt.subplot(1, 2, 2)
plt.plot(lenet_history.history['val_loss'], label='LeNet Validation Loss', linestyle='--')
plt.plot(lenet_history.history['loss'], label='LeNet Training Loss')
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(20, 8))

# Grafik Akurasi
plt.subplot(1, 2, 1)
plt.plot(vgg_history.history['val_accuracy'], label='VGG Validation Accuracy', linestyle='--')
plt.plot(vgg_history.history['accuracy'], label='VGG Training Accuracy')
plt.title('Perbandingan Akurasi Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()

# Grafik Loss
plt.subplot(1, 2, 2)
plt.plot(vgg_history.history['val_loss'], label='VGG Validation Loss', linestyle='--')
plt.plot(vgg_history.history['loss'], label='VGG Training Loss')
plt.title('Perbandingan Loss Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(20, 8))

# Grafik Akurasi
plt.subplot(1, 2, 1)
plt.plot(resnet_history.history['val_accuracy'], label='ResNet Validation Accuracy', linestyle='--')
plt.plot(resnet_history.history['accuracy'], label='ResNet Training Accuracy')
plt.title('Perbandingan Akurasi Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()

# Grafik Loss
plt.subplot(1, 2, 2)
plt.plot(resnet_history.history['val_loss'], label='ResNet Validation Loss', linestyle='--')
plt.plot(resnet_history.history['loss'], label='ResNet Training Loss')
plt.title('Perbandingan Loss Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

# --- MEMUAT MODEL DAN DATA UJI ---
print("Memuat model dan data uji...")

# Muat model terbaik
model = load_model('semaphore_lenet.h5')

try:
    X_test = np.load('X_test.npy')
    y_test = np.load('y_test.npy')
except FileNotFoundError:
    print("\nPERINGATAN: File X_test.npy atau y_test.npy tidak ditemukan.")
    print("Anda perlu menyimpan data uji dari skrip training terlebih dahulu.")
    # Jika file tidak ada, eksekusi dihentikan
    exit()

# Reshape data uji untuk prediksi
X_test_reshaped = np.expand_dims(X_test, axis=2)

print("Data berhasil dimuat.")

# --- MELAKUKAN PREDIKSI PADA DATA UJI ---
print("\nMelakukan prediksi...")
predictions = model.predict(X_test_reshaped)
# Mengambil indeks kelas dengan probabilitas tertinggi untuk setiap prediksi
predicted_classes = np.argmax(predictions, axis=1)

true_classes = y_test

# Membuat daftar label huruf dari A-Z
labels = [chr(i) for i in range(ord('A'), ord('Z') + 1)]

# --- MENAMPILKAN LAPORAN KLASIFIKASI & CONFUSION MATRIX ---

# Laporan Klasifikasi (Precision, Recall, F1-Score)
print("\nLaporan Klasifikasi Lengkap:")
print(classification_report(true_classes, predicted_classes, target_names=labels))

# Confusion Matrix untuk visualisasi kesalahan
print("\nMembuat Confusion Matrix...")
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(15, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix - Kesalahan Prediksi Model')
plt.ylabel('Kelas Sebenarnya (True Label)')
plt.xlabel('Kelas Prediksi (Predicted Label)')
plt.show()


# --- MENCARI DAN MENAMPILKAN KESALAHAN SPESIFIK ---
print("\n--- Analisis Kesalahan Spesifik ---")
error_count = 0
for i in range(len(true_classes)):
    if predicted_classes[i] != true_classes[i]:
        error_count += 1
        true_label = labels[true_classes[i]]
        predicted_label = labels[predicted_classes[i]]
        confidence = predictions[i][predicted_classes[i]] * 100
        
        print(f"{error_count}. Model salah memprediksi:")
        print(f"   > Huruf Sebenarnya: '{true_label}'")
        print(f"   > Diprediksi sebagai: '{predicted_label}' (Keyakinan: {confidence:.2f}%)")
        print("-" * 20)

if error_count == 0:
    print("Selamat! Tidak ditemukan kesalahan pada data uji.")

In [ ]:
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

# --- MEMUAT MODEL DAN DATA UJI ---
print("Memuat model dan data uji...")

# Muat model terbaik
model = load_model('semaphore_vgg_like.h5')

try:
    X_test = np.load('X_test.npy')
    y_test = np.load('y_test.npy')
except FileNotFoundError:
    print("\nPERINGATAN: File X_test.npy atau y_test.npy tidak ditemukan.")
    print("Anda perlu menyimpan data uji dari skrip training terlebih dahulu.")
    # Jika file tidak ada, eksekusi dihentikan
    exit()

# Reshape data uji untuk prediksi
X_test_reshaped = np.expand_dims(X_test, axis=2)

print("Data berhasil dimuat.")

# --- MELAKUKAN PREDIKSI PADA DATA UJI ---
print("\nMelakukan prediksi...")
predictions = model.predict(X_test_reshaped)
# Mengambil indeks kelas dengan probabilitas tertinggi untuk setiap prediksi
predicted_classes = np.argmax(predictions, axis=1)

true_classes = y_test

# Membuat daftar label huruf dari A-Z
labels = [chr(i) for i in range(ord('A'), ord('Z') + 1)]

# --- MENAMPILKAN LAPORAN KLASIFIKASI & CONFUSION MATRIX ---

# Laporan Klasifikasi (Precision, Recall, F1-Score)
print("\nLaporan Klasifikasi Lengkap:")
print(classification_report(true_classes, predicted_classes, target_names=labels))

# Confusion Matrix untuk visualisasi kesalahan
print("\nMembuat Confusion Matrix...")
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(15, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix - Kesalahan Prediksi Model')
plt.ylabel('Kelas Sebenarnya (True Label)')
plt.xlabel('Kelas Prediksi (Predicted Label)')
plt.show()


# --- MENCARI DAN MENAMPILKAN KESALAHAN SPESIFIK ---
print("\n--- Analisis Kesalahan Spesifik ---")
error_count = 0
for i in range(len(true_classes)):
    if predicted_classes[i] != true_classes[i]:
        error_count += 1
        true_label = labels[true_classes[i]]
        predicted_label = labels[predicted_classes[i]]
        confidence = predictions[i][predicted_classes[i]] * 100
        
        print(f"{error_count}. Model salah memprediksi:")
        print(f"   > Huruf Sebenarnya: '{true_label}'")
        print(f"   > Diprediksi sebagai: '{predicted_label}' (Keyakinan: {confidence:.2f}%)")
        print("-" * 20)

if error_count == 0:
    print("Selamat! Tidak ditemukan kesalahan pada data uji.")

In [ ]:
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

# --- MEMUAT MODEL DAN DATA UJI ---
print("Memuat model dan data uji...")

# Muat model terbaik
model = load_model('semaphore_resnet_like.h5')

try:
    X_test = np.load('X_test.npy')
    y_test = np.load('y_test.npy')
except FileNotFoundError:
    print("\nPERINGATAN: File X_test.npy atau y_test.npy tidak ditemukan.")
    print("Anda perlu menyimpan data uji dari skrip training terlebih dahulu.")
    # Jika file tidak ada, eksekusi dihentikan
    exit()

# Reshape data uji untuk prediksi
X_test_reshaped = np.expand_dims(X_test, axis=2)

print("Data berhasil dimuat.")

# --- MELAKUKAN PREDIKSI PADA DATA UJI ---
print("\nMelakukan prediksi...")
predictions = model.predict(X_test_reshaped)
# Mengambil indeks kelas dengan probabilitas tertinggi untuk setiap prediksi
predicted_classes = np.argmax(predictions, axis=1)

true_classes = y_test

# Membuat daftar label huruf dari A-Z
labels = [chr(i) for i in range(ord('A'), ord('Z') + 1)]

# --- MENAMPILKAN LAPORAN KLASIFIKASI & CONFUSION MATRIX ---

# Laporan Klasifikasi (Precision, Recall, F1-Score)
print("\nLaporan Klasifikasi Lengkap:")
print(classification_report(true_classes, predicted_classes, target_names=labels))

# Confusion Matrix untuk visualisasi kesalahan
print("\nMembuat Confusion Matrix...")
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(15, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix - Kesalahan Prediksi Model')
plt.ylabel('Kelas Sebenarnya (True Label)')
plt.xlabel('Kelas Prediksi (Predicted Label)')
plt.show()


# --- MENCARI DAN MENAMPILKAN KESALAHAN SPESIFIK ---
print("\n--- Analisis Kesalahan Spesifik ---")
error_count = 0
for i in range(len(true_classes)):
    if predicted_classes[i] != true_classes[i]:
        error_count += 1
        true_label = labels[true_classes[i]]
        predicted_label = labels[predicted_classes[i]]
        confidence = predictions[i][predicted_classes[i]] * 100
        
        print(f"{error_count}. Model salah memprediksi:")
        print(f"   > Huruf Sebenarnya: '{true_label}'")
        print(f"   > Diprediksi sebagai: '{predicted_label}' (Keyakinan: {confidence:.2f}%)")
        print("-" * 20)

if error_count == 0:
    print("Selamat! Tidak ditemukan kesalahan pada data uji.")

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# Muat model yang sudah dilatih
model = load_model('semaphore_lenet.h5')

# Label huruf
LABELS = [chr(i) for i in range(ord('A'), ord('Z') + 1)]

# Inisialisasi MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Konversi warna dan proses dengan MediaPipe
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Ekstraksi landmark jika terdeteksi
    if results.pose_landmarks:
        # Gambar landmark pada frame
        mp.solutions.drawing_utils.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )
        
        try:
            # Siapkan data untuk prediksi
            landmarks = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
            landmarks = np.expand_dims(landmarks, axis=0) # -> (1, 132)
            landmarks = np.expand_dims(landmarks, axis=2) # -> (1, 132, 1)
            
            # Lakukan prediksi
            prediction = model.predict(landmarks)
            predicted_class = np.argmax(prediction)
            confidence = np.max(prediction)
            
            # Tampilkan hasil
            text = f"Huruf: {LABELS[predicted_class]} ({confidence*100:.2f}%)"
            cv2.putText(image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
        except Exception as e:
            print(f"Error saat prediksi: {e}")

    cv2.imshow('Deteksi Semaphore Pramuka', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pose.close()